In [25]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs4
import re

In [33]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://tenders.ppa.gov.gh/tenders/"

def scrape_tenders():
    tenders_data = {}
    page_number = 1

    while True:
        # Construct the URL for the current page
        url = f"{BASE_URL}?page={page_number}" if page_number > 1 else BASE_URL
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch page {page_number}. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all tender links
        tender_links = soup.find_all('a', href=True)
        for link in tender_links:
            href = link['href']
            if href.startswith(BASE_URL):
                tender_name = link.text.strip()  # Extract tender name from the anchor tag's text
                if tender_name and tender_name not in tenders_data:
                    tender_info = {}

                    # Visit the tender page to extract additional information
                    tender_response = requests.get(href)
                    if tender_response.status_code != 200:
                        print(f"Failed to fetch tender page: {href}")
                        continue

                    tender_soup = BeautifulSoup(tender_response.text, 'html.parser')

                    # Extract information from <dl> elements
                    dl_elements = tender_soup.find_all('dl')
                    for dl in dl_elements:
                        dt = dl.find('dt')  # Find the <dt> (label)
                        dd = dl.find('dd')  # Find the <dd> (value)
                        if dt and dd:
                            label = dt.text.strip().replace(":", "")  # Clean up the label
                            value = dd.text.strip()  # Clean up the value
                            tender_info[label] = value

                    # Store in dictionary
                    tenders_data[tender_name] = tender_info

        # Check if there is a next page
        pagination = soup.find('a', text=str(page_number + 1))
        if not pagination:
            print("No more pages. Exiting.")
            break

        page_number += 1

    return tenders_data



In [38]:
#This is to convert our dictionary into a dataframe

def create_dataframe(tenders_data):
    rows = []
    for tender_name, details in tenders_data.items():
        row = {"Name": tender_name}
        row.update(details)  # Add other details to the row
        rows.append(row)
    df = pd.DataFrame(rows)
    df.fillna("", inplace=True)
    return df
tenders = scrape_tenders()
create_dataframe(tenders)


,Name,Tender Name,Tender Type,Package No,Post Date,Closing Date,Currency,Price of Tender Document,Tender Description,Additional Information,Source of Funds,No. of Lots,Agency,Region,District,Contact Person,Email,Telphone,Fax,Website
0,Supply Of Media Equipment,Supply of Media Equipment,NCT,GR/NYA/NCT/GDS/25/001,"22nd April, 2025","08th May, 2025",Ghana Cedi,Gh¢500.00,NO. 1\r\nIFT/Package No: GR/NYA/NCT/GDS/25/001...,1.\tThe National Youth Authority intends to ap...,GOG Consolidated Fund,1,National Youth Authority,Greater Accra,Accra Metropolitan District,The Chief Executive Officer,unknown@gmail.com,0302984293,,
1,Provision Of Cleaning/janitorial Services,Provision of Cleaning/Janitorial Services,NCT,GHS/GARH/TS/NCT/CLSERV/MAB/25/01,"21st April, 2025","30th April, 2025",Ghana Cedi,Gh¢500.00,"The Greater Accra Regional Hospital (GARH), Ri...",Tender Security - 2% of the Total Tender Value...,Internally Generated Funds,1,Greater Accra Regional Hospital,Greater Accra,Accra Metropolitan District,"Procurement Unit, 2nd Floor (In the Logistics ...",info@garh.gov.gh,0244865916,,
2,Provision Of Cleaning And Janitorial Services ...,Provision of Cleaning and Janitorial Services ...,NCT,GHS/GARH/TS/NCT/CLSERV/LBCSAB/25/02,"21st April, 2025","30th April, 2025",Ghana Cedi,Gh¢500.00,"The Greater Accra Regional Hospital (GARH), Ri...",All tenders must be accompanied by a Tender Se...,Internally Generated Funds,1,Greater Accra Regional Hospital,Greater Accra,Accra Metropolitan District,"Procurement Unit, 2nd Floor (In the Logistics ...",info@garh.gov.gh,0244865916,,
3,Provision Of Cleaning/janitorial Services For ...,Provision of Cleaning/Janitorial Services for ...,NCT,GHS/GARH/TS/NCT/CLSERV/ONMB/25/03,"21st April, 2025","30th April, 2025",Ghana Cedi,Gh¢500.00,"The Greater Accra Regional Hospital (GARH), Ri...",All tenders must be accompanied by a Tender Se...,Internally Generated Funds,1,Greater Accra Regional Hospital,Greater Accra,Accra Metropolitan District,"Procurement Unit, 2nd Floor (In the Logistics ...",info@garh.gov.gh,0244865916,,
4,Provision Of Horticulture Services For The Old...,Provision of Horticulture Services for the Old...,NCT,GHS/GARH/TS/NCT/HORTSERV/ONB/25/01,"21st April, 2025","30th April, 2025",Ghana Cedi,Gh¢500.00,"The Greater Accra Regional Hospital (GARH), Ri...",All tenders must be accompanied by a Tender Se...,Internally Generated Funds,1,Greater Accra Regional Hospital,Greater Accra,Accra Metropolitan District,"Procurement Unit, 2nd Floor (In the Logistics ...",info@garh.gov.gh,0244865916,,
5,Provision Of Security Services,Provision of Security Services,NCT,GHS/GARH/TS/NCT/SECSERV/MAB/20/01,"21st April, 2025","30th April, 2025",Ghana Cedi,Gh¢500.00,"The Greater Accra Regional Hospital (GARH), Ri...",All tenders must be accompanied by a Tender Se...,Internally Generated Funds,1,Greater Accra Regional Hospital,Greater Accra,Accra Metropolitan District,"Procurement Unit, 2nd Floor (In the Logistics ...",info@garh.gov.gh,0244865916,,
6,Supply Of Stationery,SUPPLY OF STATIONERY,NCT,STU/NCT/ FWA/GDS/2025/001,"15th April, 2025","15th May, 2025",Ghana Cedi,Gh¢400.00,SUPPLY OF STATIONERY,1.\tThe Sunyani Technical University intends t...,Internally Generated Funds,1,Sunyani Technical university,Brong-Ahafo,Sunyani Municipal,THE PROCUREMENT OFFICER\t \t\t\t\t\tSUNYANI TE...,enquiries@stu.edu.gh,"03520 27052, 24924 +233 24 546 0111",,
7,Supply Of Toners,SUPPLY OF TONERS,NCT,STU/NCT/ FWA/GDS/2025/002,"15th April, 2025","15th May, 2025",Ghana Cedi,Gh¢400.00,SUPPLY OF TONERS,1.\tThe Sunyani Technical University intends t...,Internally Generated Funds,1,Sunyani Technical university,Brong-Ahafo,Sunyani Municipal,THE PROCUREMENT OFFICER\t \t\t\t\t\tSUNYANI TE...,enquiries@stu.edu.gh,"03520 27052, 24924 +233 24 546 0111",,
8,Supply Of Drugs,SUPPLY OF DRUGS,NCT,STU/NCT/ FWA/GDS/2025/003,"15th April, 2025","15th May, 2025",Ghana Cedi,Gh¢400.00,SUPPLY OF DRUGS,1.\tThe Sunyani Technical University intends t...,Internally Generated Fund